In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((8942, 13), (8942, 13))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'AL', 'PL', 'vit', 'model', 'data', 'ALL', 'ratio', 'kw'], axis = 1) #'method',

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

,dataset,metric,value,method,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,base,5,0.000050,64
1,UCM-CLS,zeroshot-val-top1,56.84,base,5,0.000050,64
2,RSICD,image_to_text_R@1,4.76,base,5,0.000050,64
3,RSICD,image_to_text_R@5,20.05,base,5,0.000050,64
4,RSICD,image_to_text_R@10,33.88,base,5,0.000050,64
...,...,...,...,...,...,...,...
8937,Sydney,image_to_text_R@5,11.67,ours,35,0.000005,128
8938,Sydney,image_to_text_R@10,19.92,ours,35,0.000005,128
8939,Sydney,text_to_image_R@1,3.62,ours,35,0.000005,128
8940,Sydney,text_to_image_R@5,11.67,ours,35,0.000005,128


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_grouped)

value            \
                                                              mean       std   
bs  epochs method metric             dataset   lr                              
64  5      base   image_to_text_R@1  RSICD     0.000050   4.436667  0.898318   
                                     Sydney    0.000050   2.951111  0.688975   
                                     UCM       0.000050   9.903333  1.118626   
                  image_to_text_R@10 RSICD     0.000050  32.446667  1.429467   
                                     Sydney    0.000050  18.220000  1.918333   
                                     UCM       0.000050  67.855556  2.542657   
                  image_to_text_R@5  RSICD     0.000050  19.097778  0.774173   
                                     Sydney    0.000050  11.088889  1.030842   
                                     UCM       0.000050  39.331111  3.629502   
                  text_to_image_R@1  RSICD     0.000050   4.488889  0.512700   
                                     Sydney    0.000050   3.532222  0.427603   
                                     UCM       0.000050  11.094444  0.874258   
                  text_to_image_R@10 RSICD     0.000050  30.535556  1.226307   
                                     Sydney    0.000050  19.965556  1.057924   
                                     UCM       0.000050  65.283333  2.383474   
                  text_to_image_R@5  RSICD     0.000050  17.816667  0.914481   
                                     Sydney    0.000050  12.095556  1.307795   
                                     UCM       0.000050  40.427778  2.258466   
                  zeroshot-val-top1  RSICD-CLS 0.000050  63.228889  3.562897   
                                     UCM-CLS   0.000050  62.477778  3.444226   
           ours   image_to_text_R@1  RSICD     0.000005   3.966000  0.404261   
                                               0.000050   3.895556  0.551886   
                                               0.000500   0.070000  0.060000   
                                     Sydney    0.000005   2.273000  0.650522   
                                               0.000050   2.503333  0.534556   
                                               0.000500   0.222222  0.156347   
                                     UCM       0.000005   6.686000  1.457488   
                                               0.000050   7.810000  1.300327   
                                               0.000500   0.476667  0.256564   
                  image_to_text_R@10 RSICD     0.000005  26.053000  1.364169   
                                               0.000050  27.717778  2.235172   
                                               0.000500   0.978889  0.080381   
                                     Sydney    0.000005  15.936000  1.336149   
                                               0.000050  16.700000  1.422524   
                                               0.000500   2.188889  0.517215   
                                     UCM       0.000005  57.650000  2.401324   
                                               0.000050  63.382222  2.262116   
                                               0.000500   4.093333  0.717304   
                  image_to_text_R@5  RSICD     0.000005  14.853000  1.326416   
                                               0.000050  16.197778  1.804382   
                                               0.000500   0.490000  0.077942   
                                     Sydney    0.000005   8.974000  0.829112   
                                               0.000050   9.413333  1.464462   
                                               0.000500   1.118889  0.269604   
                                     UCM       0.000005  31.932000  2.380106   
                                               0.000050  35.095556  1.513672   
                                               0.000500   1.955556  0.436781   
                  text_to_image_R@1  RSICD     0.000005   3.0

In [7]:
df_grouped.to_csv('eval.csv')

In [12]:
df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'ours')] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

bs epochs method              metric    dataset       lr      value  \
                                                                    mean   
21   64      5   ours   image_to_text_R@1      RSICD  0.00005   3.895556   
101  64     10   ours   image_to_text_R@1      RSICD  0.00005   4.108889   
181  64     15   ours   image_to_text_R@1      RSICD  0.00005   4.690000   
421  64     30   ours   image_to_text_R@1      RSICD  0.00005   5.035556   
261  64     20   ours   image_to_text_R@1      RSICD  0.00005   5.096667   
341  64     25   ours   image_to_text_R@1      RSICD  0.00005   5.361111   
501  64     35   ours   image_to_text_R@1      RSICD  0.00005   5.545556   
24   64      5   ours   image_to_text_R@1     Sydney  0.00005   2.503333   
184  64     15   ours   image_to_text_R@1     Sydney  0.00005   2.571111   
264  64     20   ours   image_to_text_R@1     Sydney  0.00005   2.617778   
104  64     10   ours   image_to_text_R@1     Sydney  0.00005   3.038889   
424  64     30   ours   image_to_text_R@1     Sydney  0.00005   3.038889   
344  64     25   ours   image_to_text_R@1     Sydney  0.00005   3.062222   
504  64     35   ours   image_to_text_R@1     Sydney  0.00005   3.131111   
27   64      5   ours   image_to_text_R@1        UCM  0.00005   7.810000   
107  64     10   ours   image_to_text_R@1        UCM  0.00005   9.617778   
187  64     15   ours   image_to_text_R@1        UCM  0.00005  10.237778   
267  64     20   ours   image_to_text_R@1        UCM  0.00005  11.188889   
347  64     25   ours   image_to_text_R@1        UCM  0.00005  11.382222   
427  64     30   ours   image_to_text_R@1        UCM  0.00005  12.285556   
507  64     35   ours   image_to_text_R@1        UCM  0.00005  12.332222   
30   64      5   ours  image_to_text_R@10      RSICD  0.00005  27.717778   
110  64     10   ours  image_to_text_R@10      RSICD  0.00005  30.148889   
190  64     15   ours  image_to_text_R@10      RSICD  0.00005  31.613333   
270  64     20   ours  image_to_text_R@10      RSICD  0.00005  33.688889   
350  64     25   ours  image_to_text_R@10      RSICD  0.00005  34.502222   
430  64     30   ours  image_to_text_R@10      RSICD  0.00005  35.845556   
510  64     35   ours  image_to_text_R@10      RSICD  0.00005  36.142222   
33   64      5   ours  image_to_text_R@10     Sydney  0.00005  16.700000   
113  64     10   ours  image_to_text_R@10     Sydney  0.00005  17.125556   
273  64     20   ours  image_to_text_R@10     Sydney  0.00005  17.885556   
193  64     15   ours  image_to_text_R@10     Sydney  0.00005  17.907778   
513  64     35   ours  image_to_text_R@10     Sydney  0.00005  19.226667   
353  64     25   ours  image_to_text_R@10     Sydney  0.00005  19.360000   
433  64     30   ours  image_to_text_R@10     Sydney  0.00005  20.008889   
36   64      5   ours  image_to_text_R@10        UCM  0.00005  63.382222   
116  64     10   ours  image_to_text_R@10        UCM  0.00005  65.142222   
196  64     15   ours  image_to_text_R@10        UCM  0.00005  68.145556   
276  64     20   ours  image_to_text_R@10        UCM  0.00005  70.810000   
356  64     25   ours  image_to_text_R@10        UCM  0.00005  73.524444   
436  64     30   ours  image_to_text_R@10        UCM  0.00005  74.282222   
516  64     35   ours  image_to_text_R@10        UCM  0.00005  74.427778   
39   64      5   ours   image_to_text_R@5      RSICD  0.00005  16.197778   
119  64     10   ours   image_to_text_R@5      RSICD  0.00005  17.378889   
199  64     15   ours   image_to_text_R@5      RSICD  0.00005  18.275556   
279  64     20   ours   image_to_text_R@5      RSICD  0.00005  19.636667   
359  64     25   ours   image_to_text_R@5      RSICD  0.00005  20.492222   
439  64     30   ours   image_to_text_R@5      RSICD  0.00005  21.336667   
519  64     35   ours   image_to_text_R@5      RSICD  0.00005  21.416667   
42   64      5   ours   image_to_text_R@5     Sydney  0.00005   9.413333   
122  64     10   ours   image_to_text_R@5     Sydney 

In [14]:
### RESULTS WITHOUT MY IMPROVED FORMATTING

df_2 = df_grouped.reset_index()
# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
df_2 = df_2.sort_values(['metric', 'dataset', ('value', 'mean')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_2)

epochs              metric  bs    dataset       lr      value            \
                                                             mean       std   
361     35   image_to_text_R@1  64      RSICD  0.00005   6.350000  0.521416   
301     30   image_to_text_R@1  64      RSICD  0.00005   6.360000  0.319648   
241     25   image_to_text_R@1  64      RSICD  0.00005   6.388889  0.552188   
364     35   image_to_text_R@1  64     Sydney  0.00005   4.337778  1.074241   
304     30   image_to_text_R@1  64     Sydney  0.00005   4.358889  1.112615   
244     25   image_to_text_R@1  64     Sydney  0.00005   4.425556  0.996520   
307     30   image_to_text_R@1  64        UCM  0.00005  13.810000  1.697233   
247     25   image_to_text_R@1  64        UCM  0.00005  14.138889  2.644463   
367     35   image_to_text_R@1  64        UCM  0.00005  14.428889  1.961112   
250     25  image_to_text_R@10  64      RSICD  0.00005  38.084444  1.517927   
310     30  image_to_text_R@10  64      RSICD  0.00005  38.684444  1.521998   
370     35  image_to_text_R@10  64      RSICD  0.00005  38.888889  1.373239   
253     25  image_to_text_R@10  64     Sydney  0.00005  22.757778  1.512546   
313     30  image_to_text_R@10  64     Sydney  0.00005  22.757778  1.471299   
373     35  image_to_text_R@10  64     Sydney  0.00005  23.542222  1.543412   
256     25  image_to_text_R@10  64        UCM  0.00005  75.380000  2.245033   
376     35  image_to_text_R@10  64        UCM  0.00005  75.760000  2.211741   
316     30  image_to_text_R@10  64        UCM  0.00005  76.188889  2.409307   
259     25   image_to_text_R@5  64      RSICD  0.00005  23.932222  0.959841   
319     30   image_to_text_R@5  64      RSICD  0.00005  23.942222  1.059691   
379     35   image_to_text_R@5  64      RSICD  0.00005  24.368889  0.875549   
262     25   image_to_text_R@5  64     Sydney  0.00005  14.285556  1.578006   
322     30   image_to_text_R@5  64     Sydney  0.00005  14.622222  1.667459   
382     35   image_to_text_R@5  64     Sydney  0.00005  15.067778  1.862551   
265     25   image_to_text_R@5  64        UCM  0.00005  48.570000  3.156501   
385     35   image_to_text_R@5  64        UCM  0.00005  50.003333  3.977612   
325     30   image_to_text_R@5  64        UCM  0.00005  50.098889  3.304476   
388     35   text_to_image_R@1  64      RSICD  0.00005   6.785556  0.802217   
328     30   text_to_image_R@1  64      RSICD  0.00005   6.807778  0.789252   
268     25   text_to_image_R@1  64      RSICD  0.00005   7.223333  0.546786   
271     25   text_to_image_R@1  64     Sydney  0.00005   5.365556  0.892106   
331     30   text_to_image_R@1  64     Sydney  0.00005   5.498889  1.258476   
391     35   text_to_image_R@1  64     Sydney  0.00005   5.590000  1.132597   
274     25   text_to_image_R@1  64        UCM  0.00005  13.095556  1.333455   
334     30   text_to_image_R@1  64        UCM  0.00005  13.668889  1.461245   
394     35   text_to_image_R@1  64        UCM  0.00005  14.048889  1.209509   
277     25  text_to_image_R@10  64      RSICD  0.00005  38.278889  1.401717   
337     30  text_to_image_R@10  64      RSICD  0.00005  38.298889  1.309871   
397     35  text_to_image_R@10  64      RSICD  0.00005  38.340000  1.504859   
280     25  text_to_image_R@10  64     Sydney  0.00005  25.418889  1.597901   
340     30  text_to_image_R@10  64     Sydney  0.00005  26.247778  1.988597   
400     35  text_to_image_R@10  64     Sydney  0.00005  26.291111  1.986281   
283     25  text_to_image_R@10  64        UCM  0.00005  74.998889  2.956850   
343     30  text_to_image_R@10  64        UCM  0.00005  75.044444  3.095295   
403     35  text_to_image_R@10  64        UCM  0.00005  75.283333  2.384513   
286     25   text_to_image_R@5  64      RSICD  0.00005  24.023333  0.891838   
346     30   text_to_image_R@5  64      RSICD  0.00005  24.113333  0.685894   
406     35   text_to_image_R@5  64      RSICD  0.00005  24.286667  0.973614   
289     25   text_to_image_R@5  64     Sydney  0.00005